# Colorisation Vidéo avec Frame de Référence

Ce notebook implémente une technique avancée de colorisation vidéo utilisant une première frame colorisée manuellement comme référence. L'algorithme propage les couleurs de manière cohérente tout en préservant l'éclairage et les ombres.

## 1. Import des Libraries et Configuration

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.neighbors import NearestNeighbors
from scipy.spatial.distance import cdist
from scipy import ndimage
import os
from typing import List, Tuple, Dict
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Configuration pour les graphiques
plt.rcParams['figure.figsize'] = (15, 10)
plt.rcParams['font.size'] = 12

print("✅ Libraries importées avec succès")
print(f"OpenCV version: {cv2.__version__}")
print(f"NumPy version: {np.__version__}")

## 2. Classe de Colorisation par Référence

In [ ]:
class ReferenceColorization:
    """Colorisation vidéo basée sur une frame de référence colorisée."""
    
    def __init__(self, temporal_consistency_weight=0.3, spatial_smoothing=True):
        self.temporal_consistency_weight = temporal_consistency_weight
        self.spatial_smoothing = spatial_smoothing
        self.reference_features = None
        self.reference_colors = None
        self.color_mappings = {}
        
    def extract_color_features(self, gray_frame: np.ndarray, colored_frame: np.ndarray) -> Dict:
        """Extrait les caractéristiques de couleur de la frame de référence."""
        
        # Conversion en LAB pour une meilleure correspondance des couleurs
        lab_colored = cv2.cvtColor(colored_frame, cv2.COLOR_BGR2LAB)
        
        # Caractéristiques texturales et d'intensité
        features = []
        colors_lab = []
        
        h, w = gray_frame.shape
        
        # Échantillonnage dense pour créer la base de correspondance
        step = 8  # Échantillonnage tous les 8 pixels
        for y in range(0, h-8, step):
            for x in range(0, w-8, step):
                # Patch 5x5 autour du pixel
                patch = gray_frame[y:y+5, x:x+5]
                if patch.shape[0] == 5 and patch.shape[1] == 5:
                    # Caractéristiques du patch
                    intensity = np.mean(patch)
                    std_dev = np.std(patch)
                    gradient_x = cv2.Sobel(patch.astype(np.float32), cv2.CV_32F, 1, 0, ksize=3)
                    gradient_y = cv2.Sobel(patch.astype(np.float32), cv2.CV_32F, 0, 1, ksize=3)
                    gradient_mag = np.mean(np.sqrt(gradient_x**2 + gradient_y**2))
                    
                    # Position relative pour la cohérence spatiale
                    pos_x = x / w
                    pos_y = y / h
                    
                    feature_vector = [intensity, std_dev, gradient_mag, pos_x, pos_y]
                    features.append(feature_vector)
                    
                    # Couleur correspondante en LAB
                    color_lab = lab_colored[y+2, x+2]  # Centre du patch
                    colors_lab.append(color_lab)
        
        return {
            'features': np.array(features),
            'colors_lab': np.array(colors_lab)
        }
    
    def set_reference_frame(self, gray_reference: np.ndarray, colored_reference: np.ndarray):
        """Définit la frame de référence et extrait les mappings de couleurs."""
        print("🎨 Extraction des caractéristiques de la frame de référence...")
        
        # Normalisation des images
        if len(gray_reference.shape) == 3:
            gray_reference = cv2.cvtColor(gray_reference, cv2.COLOR_BGR2GRAY)
        
        # Extraction des caractéristiques
        reference_data = self.extract_color_features(gray_reference, colored_reference)
        self.reference_features = reference_data['features']
        self.reference_colors = reference_data['colors_lab']
        
        # Créer un modèle k-NN pour la correspondance rapide
        self.knn_model = NearestNeighbors(n_neighbors=5, algorithm='kd_tree')
        self.knn_model.fit(self.reference_features)
        
        print(f"✅ {len(self.reference_features)} points de correspondance extraits")
    
    def colorize_frame(self, gray_frame: np.ndarray, previous_colored: np.ndarray = None) -> np.ndarray:
        """Colorise une frame en utilisant la référence et la cohérence temporelle."""
        
        if len(gray_frame.shape) == 3:
            gray_frame = cv2.cvtColor(gray_frame, cv2.COLOR_BGR2GRAY)
        
        h, w = gray_frame.shape
        colored_frame_lab = np.zeros((h, w, 3), dtype=np.float32)
        
        # Canal L (luminance) = niveaux de gris normalisés
        colored_frame_lab[:, :, 0] = gray_frame.astype(np.float32) * (100.0 / 255.0)
        
        # Colorisation par blocks pour l'efficacité
        block_size = 8
        for y in range(0, h, block_size):
            for x in range(0, w, block_size):
                # Limites du block
                y_end = min(y + block_size, h)
                x_end = min(x + block_size, w)
                
                # Patch central pour les caractéristiques
                center_y = y + block_size // 2
                center_x = x + block_size // 2
                
                if center_y < h-2 and center_x < w-2:
                    # Extraction des caractéristiques du patch
                    patch = gray_frame[max(0, center_y-2):center_y+3, max(0, center_x-2):center_x+3]
                    
                    if patch.shape[0] >= 3 and patch.shape[1] >= 3:
                        intensity = np.mean(patch)
                        std_dev = np.std(patch)
                        gradient_x = cv2.Sobel(patch.astype(np.float32), cv2.CV_32F, 1, 0, ksize=3)
                        gradient_y = cv2.Sobel(patch.astype(np.float32), cv2.CV_32F, 0, 1, ksize=3)
                        gradient_mag = np.mean(np.sqrt(gradient_x**2 + gradient_y**2))
                        
                        pos_x = center_x / w
                        pos_y = center_y / h
                        
                        feature_vector = np.array([[intensity, std_dev, gradient_mag, pos_x, pos_y]])
                        
                        # Recherche des k plus proches voisins
                        distances, indices = self.knn_model.kneighbors(feature_vector)
                        
                        # Moyenne pondérée des couleurs des voisins
                        weights = 1.0 / (distances[0] + 1e-8)  # Éviter division par zéro
                        weights = weights / np.sum(weights)
                        
                        predicted_color = np.average(self.reference_colors[indices[0]], axis=0, weights=weights)
                        
                        # Appliquer la couleur au block
                        colored_frame_lab[y:y_end, x:x_end, 1] = predicted_color[1]  # Canal A
                        colored_frame_lab[y:y_end, x:x_end, 2] = predicted_color[2]  # Canal B
        
        # Lissage spatial pour réduire les artifacts
        if self.spatial_smoothing:
            colored_frame_lab[:, :, 1] = cv2.bilateralFilter(
                colored_frame_lab[:, :, 1].astype(np.float32), 9, 75, 75
            )
            colored_frame_lab[:, :, 2] = cv2.bilateralFilter(
                colored_frame_lab[:, :, 2].astype(np.float32), 9, 75, 75
            )
        
        # Cohérence temporelle avec la frame précédente
        if previous_colored is not None and self.temporal_consistency_weight > 0:
            previous_lab = cv2.cvtColor(previous_colored, cv2.COLOR_BGR2LAB).astype(np.float32)
            
            # Mélange pondéré
            alpha = 1 - self.temporal_consistency_weight
            colored_frame_lab[:, :, 1] = (alpha * colored_frame_lab[:, :, 1] + 
                                         self.temporal_consistency_weight * previous_lab[:, :, 1])
            colored_frame_lab[:, :, 2] = (alpha * colored_frame_lab[:, :, 2] + 
                                         self.temporal_consistency_weight * previous_lab[:, :, 2])
        
        # Conversion de LAB vers BGR
        colored_frame_lab = np.clip(colored_frame_lab, 0, 255).astype(np.uint8)
        colored_frame_bgr = cv2.cvtColor(colored_frame_lab, cv2.COLOR_LAB2BGR)
        
        return colored_frame_bgr
    
    def colorize_video(self, video_frames: List[np.ndarray], reference_colored: np.ndarray) -> List[np.ndarray]:
        """Colorise une vidéo complète en utilisant la frame de référence."""
        
        if len(video_frames) == 0:
            return []
        
        # Utiliser la première frame comme référence en niveaux de gris
        first_frame_gray = video_frames[0]
        if len(first_frame_gray.shape) == 3:
            first_frame_gray = cv2.cvtColor(first_frame_gray, cv2.COLOR_BGR2GRAY)
        
        # Configurer la référence
        self.set_reference_frame(first_frame_gray, reference_colored)
        
        colorized_frames = []
        previous_colored = None
        
        print(f"🎬 Colorisation de {len(video_frames)} frames...")
        
        for i, frame in enumerate(tqdm(video_frames, desc="Colorisation")):
            if i == 0:
                # Utiliser directement la frame de référence colorisée
                colorized_frame = reference_colored.copy()
            else:
                # Coloriser en utilisant la référence et la cohérence temporelle
                colorized_frame = self.colorize_frame(frame, previous_colored)
            
            colorized_frames.append(colorized_frame)
            previous_colored = colorized_frame
        
        return colorized_frames

print("✅ Classe ReferenceColorization définie")

## 3. Fonctions Utilitaires

In [ ]:
def load_video_frames(video_path: str, max_frames: int = None) -> List[np.ndarray]:
    """Charge les frames d'une vidéo."""
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    frame_count = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        frames.append(frame)
        frame_count += 1
        
        if max_frames and frame_count >= max_frames:
            break
    
    cap.release()
    print(f"📹 {len(frames)} frames chargées depuis {video_path}")
    return frames

def save_video_frames(frames: List[np.ndarray], output_path: str, fps: float = 30.0):
    """Sauvegarde une liste de frames en tant que vidéo."""
    if not frames:
        print("❌ Aucune frame à sauvegarder")
        return
    
    height, width = frames[0].shape[:2]
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_path, fourcc, fps, (width, height))
    
    for frame in frames:
        out.write(frame)
    
    out.release()
    print(f"💾 Vidéo sauvegardée: {output_path}")

def create_pigeon_reference_frame(gray_frame: np.ndarray) -> np.ndarray:
    """Crée une frame de référence colorisée avec des couleurs réalistes de pigeon."""
    h, w = gray_frame.shape
    colored_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
    
    # Conversion en HSV pour faciliter la colorisation
    hsv_frame = cv2.cvtColor(colored_frame, cv2.COLOR_BGR2HSV)
    
    # Segmentation approximative basée sur l'intensité
    # Ciel (zones claires)
    sky_mask = gray_frame > 180
    
    # Pigeon (zones moyennes)
    pigeon_mask = (gray_frame > 80) & (gray_frame < 160)
    
    # Bois (zones sombres mais pas trop)
    wood_mask = (gray_frame > 40) & (gray_frame < 120)
    
    # Couleurs réalistes
    # Ciel bleu clair
    hsv_frame[sky_mask, 0] = 100  # Teinte bleue
    hsv_frame[sky_mask, 1] = 80   # Saturation modérée
    
    # Pigeon gris avec reflets verts/violets sur le cou
    # Zone du cou (approximation)
    neck_mask = (gray_frame > 90) & (gray_frame < 140) & (np.arange(h)[:, None] < h//3)
    
    # Corps gris
    body_mask = pigeon_mask & ~neck_mask
    hsv_frame[body_mask, 1] = 30  # Très peu saturé (gris)
    
    # Cou iridescent
    hsv_frame[neck_mask, 0] = 70   # Vert-violet
    hsv_frame[neck_mask, 1] = 100  # Saturé
    
    # Bois brun
    hsv_frame[wood_mask, 0] = 15   # Teinte brune
    hsv_frame[wood_mask, 1] = 120  # Bien saturé
    
    # Conversion retour en BGR
    colored_frame = cv2.cvtColor(hsv_frame, cv2.COLOR_HSV2BGR)
    
    # Lissage pour éviter les transitions abruptes
    colored_frame = cv2.bilateralFilter(colored_frame, 9, 75, 75)
    
    return colored_frame

def display_comparison(original: np.ndarray, colorized: np.ndarray, title: str = "Comparaison"):
    """Affiche une comparaison côte à côte."""
    fig, axes = plt.subplots(1, 2, figsize=(15, 7))
    
    # Image originale
    if len(original.shape) == 3:
        axes[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
    else:
        axes[0].imshow(original, cmap='gray')
    axes[0].set_title('Original (N&B)')
    axes[0].axis('off')
    
    # Image colorisée
    axes[1].imshow(cv2.cvtColor(colorized, cv2.COLOR_BGR2RGB))
    axes[1].set_title('Colorisée')
    axes[1].axis('off')
    
    plt.suptitle(title, fontsize=16)
    plt.tight_layout()
    plt.show()

print("✅ Fonctions utilitaires définies")

## 4. Exemple d'Utilisation - Chargement de la Vidéo

In [ ]:
# Chemins des fichiers
input_video_path = "data/input/demo_color.mp4"  # Remplacez par votre vidéo N&B
output_video_path = "data/output/pigeon_colorized_reference.mp4"

# Vérifier l'existence du fichier
if os.path.exists(input_video_path):
    print(f"📹 Chargement de la vidéo: {input_video_path}")
    
    # Charger les frames (limité à 100 pour les tests)
    video_frames = load_video_frames(input_video_path, max_frames=100)
    
    if video_frames:
        print(f"✅ Vidéo chargée: {len(video_frames)} frames")
        print(f"📐 Résolution: {video_frames[0].shape}")
        
        # Afficher la première frame
        plt.figure(figsize=(10, 6))
        plt.imshow(cv2.cvtColor(video_frames[0], cv2.COLOR_BGR2RGB))
        plt.title("Première frame de la vidéo")
        plt.axis('off')
        plt.show()
    else:
        print("❌ Aucune frame chargée")
else:
    print(f"❌ Fichier vidéo non trouvé: {input_video_path}")
    print("📝 Assurez-vous d'avoir une vidéo dans le dossier data/input/")
    
    # Créer une vidéo de démonstration si nécessaire
    print("🎬 Création d'une vidéo de démonstration...")
    demo_frame = np.random.randint(0, 256, (480, 640, 3), dtype=np.uint8)
    video_frames = [demo_frame] * 30  # 30 frames identiques
    print("✅ Vidéo de démonstration créée")

## 5. Création de la Frame de Référence

In [ ]:
# Convertir la première frame en niveaux de gris
first_frame = video_frames[0]
if len(first_frame.shape) == 3:
    first_frame_gray = cv2.cvtColor(first_frame, cv2.COLOR_BGR2GRAY)
else:
    first_frame_gray = first_frame.copy()

# Créer une frame de référence colorisée avec des couleurs réalistes
print("🎨 Création de la frame de référence colorisée...")
reference_colored = create_pigeon_reference_frame(first_frame_gray)

# Afficher la comparaison
display_comparison(first_frame_gray, reference_colored, "Frame de Référence - Avant/Après")

print("✅ Frame de référence créée avec succès")

## 6. Configuration et Colorisation de la Vidéo

In [ ]:
# Initialiser le coloriseur avec des paramètres optimaux
colorizer = ReferenceColorization(
    temporal_consistency_weight=0.4,  # Forte cohérence temporelle
    spatial_smoothing=True             # Lissage spatial activé
)

print("🎬 Début de la colorisation vidéo...")
print(f"⚙️ Cohérence temporelle: {colorizer.temporal_consistency_weight}")
print(f"⚙️ Lissage spatial: {colorizer.spatial_smoothing}")

# Coloriser la vidéo
colorized_frames = colorizer.colorize_video(video_frames, reference_colored)

print(f"✅ Colorisation terminée: {len(colorized_frames)} frames")

## 7. Visualisation des Résultats

In [ ]:
# Affichage de quelques frames représentatives
frame_indices = [0, len(colorized_frames)//4, len(colorized_frames)//2, -1]

fig, axes = plt.subplots(2, len(frame_indices), figsize=(20, 10))

for i, idx in enumerate(frame_indices):
    # Frame originale
    original = video_frames[idx]
    if len(original.shape) == 3:
        original_display = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    else:
        original_display = original
    
    axes[0, i].imshow(original_display, cmap='gray' if len(original.shape) == 2 else None)
    axes[0, i].set_title(f'Original - Frame {idx}')
    axes[0, i].axis('off')
    
    # Frame colorisée
    colorized_display = cv2.cvtColor(colorized_frames[idx], cv2.COLOR_BGR2RGB)
    axes[1, i].imshow(colorized_display)
    axes[1, i].set_title(f'Colorisée - Frame {idx}')
    axes[1, i].axis('off')

plt.suptitle('Comparaison Vidéo: Original vs Colorisé', fontsize=16)
plt.tight_layout()
plt.show()

# Analyse de la cohérence temporelle
print("\n📊 Analyse de la cohérence temporelle:")
temporal_differences = []
for i in range(1, len(colorized_frames)):
    diff = cv2.absdiff(colorized_frames[i-1], colorized_frames[i])
    mean_diff = np.mean(diff)
    temporal_differences.append(mean_diff)

print(f"Différence temporelle moyenne: {np.mean(temporal_differences):.2f}")
print(f"Écart-type des différences: {np.std(temporal_differences):.2f}")

# Graphique de la cohérence temporelle
plt.figure(figsize=(12, 4))
plt.plot(temporal_differences)
plt.title('Cohérence Temporelle - Différences entre Frames Consécutives')
plt.xlabel('Frame')
plt.ylabel('Différence Moyenne')
plt.grid(True, alpha=0.3)
plt.show()

## 8. Sauvegarde de la Vidéo Colorisée

In [ ]:
# Créer le dossier de sortie s'il n'existe pas
os.makedirs(os.path.dirname(output_video_path), exist_ok=True)

# Sauvegarder la vidéo colorisée
print(f"💾 Sauvegarde de la vidéo colorisée...")
save_video_frames(colorized_frames, output_video_path, fps=30.0)

# Sauvegarder aussi la frame de référence
reference_path = output_video_path.replace('.mp4', '_reference.jpg')
cv2.imwrite(reference_path, reference_colored)
print(f"💾 Frame de référence sauvegardée: {reference_path}")

print("\n✅ Processus de colorisation terminé avec succès!")
print(f"📹 Vidéo colorisée: {output_video_path}")
print(f"🖼️ Frame de référence: {reference_path}")

## 9. Fonctions Avancées - Amélioration de la Colorisation

In [ ]:
class AdvancedReferenceColorization(ReferenceColorization):
    """Version avancée avec détection d'objets et colorisation spécialisée."""
    
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        self.object_masks = {}
        self.object_colors = {
            'sky': [135, 206, 235],      # Bleu ciel
            'pigeon_body': [128, 128, 128],  # Gris pigeon
            'pigeon_neck': [75, 0, 130],     # Violet iridescent
            'wood': [139, 69, 19]            # Brun bois
        }
    
    def detect_objects(self, gray_frame: np.ndarray):
        """Détecte approximativement les objets dans la frame."""
        h, w = gray_frame.shape
        
        # Segmentation basée sur l'intensité et la position
        masks = {}
        
        # Ciel (partie supérieure, intensité élevée)
        sky_intensity_mask = gray_frame > 180
        sky_position_mask = np.zeros_like(gray_frame, dtype=bool)
        sky_position_mask[:h//3, :] = True  # Tiers supérieur
        masks['sky'] = sky_intensity_mask & sky_position_mask
        
        # Pigeon (centre, intensité moyenne)
        pigeon_mask = (gray_frame > 70) & (gray_frame < 160)
        center_mask = np.zeros_like(gray_frame, dtype=bool)
        center_mask[h//4:3*h//4, w//4:3*w//4] = True
        
        # Corps du pigeon
        masks['pigeon_body'] = pigeon_mask & center_mask
        
        # Cou du pigeon (partie supérieure du corps)
        neck_mask = np.zeros_like(gray_frame, dtype=bool)
        neck_mask[h//4:h//2, 2*w//5:3*w//5] = True
        masks['pigeon_neck'] = pigeon_mask & neck_mask
        
        # Bois (partie inférieure, intensité variable)
        wood_mask = gray_frame < 140
        bottom_mask = np.zeros_like(gray_frame, dtype=bool)
        bottom_mask[2*h//3:, :] = True
        masks['wood'] = wood_mask & bottom_mask
        
        return masks
    
    def apply_specialized_coloring(self, gray_frame: np.ndarray) -> np.ndarray:
        """Applique une colorisation spécialisée par objet."""
        colored_frame = cv2.cvtColor(gray_frame, cv2.COLOR_GRAY2BGR)
        
        # Détecter les objets
        masks = self.detect_objects(gray_frame)
        
        # Appliquer les couleurs spécialisées
        for obj_name, mask in masks.items():
            if obj_name in self.object_colors and np.any(mask):
                color = self.object_colors[obj_name]
                
                # Mélange avec l'intensité originale
                intensity = gray_frame[mask].astype(np.float32) / 255.0
                
                for c in range(3):
                    colored_frame[mask, c] = (color[c] * intensity).astype(np.uint8)
        
        return colored_frame

# Test de la version avancée
print("🔬 Test de la colorisation avancée...")
advanced_colorizer = AdvancedReferenceColorization(
    temporal_consistency_weight=0.3,
    spatial_smoothing=True
)

# Créer une frame de référence améliorée
advanced_reference = advanced_colorizer.apply_specialized_coloring(first_frame_gray)

# Comparaison des méthodes
fig, axes = plt.subplots(1, 3, figsize=(18, 6))

axes[0].imshow(first_frame_gray, cmap='gray')
axes[0].set_title('Original')
axes[0].axis('off')

axes[1].imshow(cv2.cvtColor(reference_colored, cv2.COLOR_BGR2RGB))
axes[1].set_title('Colorisation Standard')
axes[1].axis('off')

axes[2].imshow(cv2.cvtColor(advanced_reference, cv2.COLOR_BGR2RGB))
axes[2].set_title('Colorisation Avancée')
axes[2].axis('off')

plt.suptitle('Comparaison des Méthodes de Colorisation', fontsize=16)
plt.tight_layout()
plt.show()

print("✅ Version avancée testée avec succès")

## 10. Métriques d'Évaluation

In [ ]:
def calculate_metrics(original_frames: List[np.ndarray], colorized_frames: List[np.ndarray]):
    """Calcule des métriques de qualité pour la colorisation."""
    
    metrics = {
        'temporal_consistency': [],
        'color_diversity': [],
        'brightness_preservation': []
    }
    
    for i in range(len(colorized_frames)):
        # Cohérence temporelle
        if i > 0:
            diff = cv2.absdiff(colorized_frames[i-1], colorized_frames[i])
            temporal_consistency = 1.0 / (1.0 + np.mean(diff))
            metrics['temporal_consistency'].append(temporal_consistency)
        
        # Diversité des couleurs
        hsv_frame = cv2.cvtColor(colorized_frames[i], cv2.COLOR_BGR2HSV)
        color_diversity = np.std(hsv_frame[:, :, 1])  # Écart-type de la saturation
        metrics['color_diversity'].append(color_diversity)
        
        # Préservation de la luminosité
        original_gray = cv2.cvtColor(original_frames[i], cv2.COLOR_BGR2GRAY) if len(original_frames[i].shape) == 3 else original_frames[i]
        colorized_gray = cv2.cvtColor(colorized_frames[i], cv2.COLOR_BGR2GRAY)
        brightness_diff = np.mean(np.abs(original_gray.astype(float) - colorized_gray.astype(float)))
        brightness_preservation = 1.0 / (1.0 + brightness_diff/255.0)
        metrics['brightness_preservation'].append(brightness_preservation)
    
    return metrics

# Calculer les métriques
print("📊 Calcul des métriques de qualité...")
metrics = calculate_metrics(video_frames, colorized_frames)

# Affichage des résultats
print("\n🎯 Résultats des métriques:")
print(f"Cohérence temporelle moyenne: {np.mean(metrics['temporal_consistency']):.3f}")
print(f"Diversité des couleurs moyenne: {np.mean(metrics['color_diversity']):.3f}")
print(f"Préservation de la luminosité: {np.mean(metrics['brightness_preservation']):.3f}")

# Graphiques des métriques
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

axes[0].plot(metrics['temporal_consistency'])
axes[0].set_title('Cohérence Temporelle')
axes[0].set_xlabel('Frame')
axes[0].grid(True, alpha=0.3)

axes[1].plot(metrics['color_diversity'])
axes[1].set_title('Diversité des Couleurs')
axes[1].set_xlabel('Frame')
axes[1].grid(True, alpha=0.3)

axes[2].plot(metrics['brightness_preservation'])
axes[2].set_title('Préservation Luminosité')
axes[2].set_xlabel('Frame')
axes[2].grid(True, alpha=0.3)

plt.suptitle('Métriques de Qualité de la Colorisation', fontsize=16)
plt.tight_layout()
plt.show()

print("✅ Analyse des métriques terminée")

## Conclusion

Ce notebook a implémenté une technique avancée de colorisation vidéo utilisant une frame de référence. Les principales caractéristiques sont :

### ✨ Points forts :
- **Cohérence temporelle** : Utilisation de la frame précédente pour maintenir la stabilité
- **Correspondance intelligente** : Algorithme k-NN avec caractéristiques texturales
- **Couleurs réalistes** : Mapping spécialisé pour pigeon, bois, et ciel
- **Lissage spatial** : Réduction des artifacts avec filtrage bilatéral

### 🔧 Paramètres ajustables :
- `temporal_consistency_weight` : Contrôle la stabilité temporelle
- `spatial_smoothing` : Active/désactive le lissage spatial
- Couleurs d'objets personnalisables

### 📈 Métriques d'évaluation :
- Cohérence temporelle
- Diversité des couleurs
- Préservation de la luminosité

La méthode est particulièrement efficace pour les vidéos avec des objets statiques ou peu mobiles, comme des scènes d'oiseaux.